In [1]:
import numpy as np
import pandas as pd
import cv2

import os 



In [94]:
df = pd.read_csv(r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_seen_training_siamese.csv", index_col = 0)
#print(df)
df = np.array(df)
print(df)

def datagen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    #print(df)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\seen-dataset\\TrainingSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\seen-dataset\\TrainingSet",df[j][1]),0)
            y = df[j][2]
            #print(loaded_image_1.shape)
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            #print(j)
            if c == batch_size:
                #print(label)
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]
                
def valgen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\seen-dataset\\ValidationSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\seen-dataset\\ValidationSet",df[j][1]),0)
            y = df[j][2]
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            if c == batch_size:
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]

[['0725a_num4.png' '0725a_num5.png' 1 ... nan nan nan]
 ['0725a_num4.png' '0725b_num3.png' 1 ... nan nan nan]
 ['0725a_num4.png' '0725c_num1.png' 1 ... nan nan nan]
 ...
 ['1469b_num3.png' '1162c_num5.png' 0 ... nan nan nan]
 ['1469b_num3.png' '0566b_num5.png' 0 ... nan nan nan]
 ['1469b_num3.png' '0753c_num1.png' 0 ... nan nan nan]]


In [44]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D, Flatten, Dense
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

imDim = 64
input_shape  = (imDim,imDim,1)
inp_img = Input(shape = (imDim,imDim,1), name = 'ImageInput')
model = inp_img

#     model = Input(shape=(imDim,imDim,1))
#     model.add(Input(shape = (imDim,imDim,1), name = 'FeatureNet_ImageInput'))
model = Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='valid')(model)
#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model = MaxPooling2D((2,2), padding='valid')(model)
model = Conv2D(64, (3, 3), activation='relu',padding='valid')(model)
#     model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
model = Conv2D(128, (3, 3), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(1, (3, 3), activation='relu',padding='same'))
#     model.add(Conv2D(2, (3, 3), activation='relu',padding='same'))

model = Conv2D(256, (1, 1), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)

model = Conv2D(64, (1, 1), activation='relu',padding='valid')(model)
# model = MaxPooling2D((2,2),padding='valid')(model)
model = Flatten()(model)

feat = Model(inputs=[inp_img], outputs=[model],name = 'Feat_Model')
feat.summary()

left_img = Input(shape = (imDim,imDim,1), name = 'left_img')
right_img = Input(shape = (imDim,imDim,1), name = 'right_img')

left_feats = feat(left_img)
right_feats = feat(right_img)

from keras.layers import concatenate
import random

merged_feats = concatenate([left_feats, right_feats], name = 'concat_feats')
merged_feats = Dense(1024, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(4, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(1, activation = 'sigmoid')(merged_feats)
similarity_model = Model(inputs = [left_img, right_img], outputs = [merged_feats], name = 'Similarity_Model')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 6, 6, 128)         0         
__________

In [95]:
similarity_model.compile(loss="binary_crossentropy",optimizer='Adam',metrics=['acc'])

In [96]:
similarity_model.fit_generator(
        (datagen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_seen_training_siamese.csv")),
        steps_per_epoch = 113790/32,
        epochs=20,
        validation_data = (valgen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_seen_validation_siamese.csv")),
        validation_steps = 906/32
        )

Epoch 1/20
3556/3555 [==============================] - 484s 136ms/step - loss: 0.1314 - acc: 0.9804 - val_loss: 2.3422 - val_acc: 0.8276
Epoch 2/20
3556/3555 [==============================] - 343s 96ms/step - loss: 0.0458 - acc: 0.9931 - val_loss: 2.3748 - val_acc: 0.8276
Epoch 3/20
3556/3555 [==============================] - 340s 96ms/step - loss: 0.0390 - acc: 0.9940 - val_loss: 2.2964 - val_acc: 0.8405
Epoch 4/20
3556/3555 [==============================] - 365s 103ms/step - loss: 0.0330 - acc: 0.9947 - val_loss: 2.2512 - val_acc: 0.8470
Epoch 5/20
3556/3555 [==============================] - 267s 75ms/step - loss: 0.0455 - acc: 0.9936 - val_loss: 1.9591 - val_acc: 0.8599
Epoch 6/20
3556/3555 [==============================] - 140s 39ms/step - loss: 0.0394 - acc: 0.9945 - val_loss: 1.9457 - val_acc: 0.8534
Epoch 7/20
3556/3555 [==============================] - 143s 40ms/step - loss: 0.0320 - acc: 0.9953 - val_loss: 1.2989 - val_acc: 0.9052
Epoch 8/20
3556/3555 [=================

In [2]:
df = pd.read_csv(r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_shuffled_training_siamese.csv", index_col = 0)
#print(df)
df = np.array(df)
print(df)

def datagen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    #print(df)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\shuffled-dataset\\TrainingSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\shuffled-dataset\\TrainingSet",df[j][1]),0)
            y = df[j][2]
            #print(loaded_image_1.shape)
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            #print(j)
            if c == batch_size:
                #print(label)
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]
                
def valgen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\shuffled-dataset\\ValidationSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\shuffled-dataset\\ValidationSet",df[j][1]),0)
            y = df[j][2]
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            if c == batch_size:
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]

[['0725a_num4.png' '0725a_num5.png' 1]
 ['0725a_num4.png' '0725b_num3.png' 1]
 ['0725a_num4.png' '0725c_num1.png' 1]
 ...
 ['1469b_num3.png' '0620c_num1.png' 0]
 ['1469b_num3.png' '1352b_num2.png' 0]
 ['1469b_num3.png' '0608b_num4.png' 0]]


In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D, Flatten, Dense
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

imDim = 64
input_shape  = (imDim,imDim,1)
inp_img = Input(shape = (imDim,imDim,1), name = 'ImageInput')
model = inp_img

#     model = Input(shape=(imDim,imDim,1))
#     model.add(Input(shape = (imDim,imDim,1), name = 'FeatureNet_ImageInput'))
model = Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='valid')(model)
#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model = MaxPooling2D((2,2), padding='valid')(model)
model = Conv2D(64, (3, 3), activation='relu',padding='valid')(model)
#     model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
model = Conv2D(128, (3, 3), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(1, (3, 3), activation='relu',padding='same'))
#     model.add(Conv2D(2, (3, 3), activation='relu',padding='same'))

model = Conv2D(256, (1, 1), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)

model = Conv2D(64, (1, 1), activation='relu',padding='valid')(model)
# model = MaxPooling2D((2,2),padding='valid')(model)
model = Flatten()(model)

feat = Model(inputs=[inp_img], outputs=[model],name = 'Feat_Model')
feat.summary()

left_img = Input(shape = (imDim,imDim,1), name = 'left_img')
right_img = Input(shape = (imDim,imDim,1), name = 'right_img')

left_feats = feat(left_img)
right_feats = feat(right_img)

from keras.layers import concatenate
import random

merged_feats = concatenate([left_feats, right_feats], name = 'concat_feats')
merged_feats = Dense(1024, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(4, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(1, activation = 'sigmoid')(merged_feats)
similarity_model_2 = Model(inputs = [left_img, right_img], outputs = [merged_feats], name = 'Similarity_Model_2')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 128)         0         
__________

In [ ]:
similarity_model_2.compile(loss="binary_crossentropy",optimizer='Adam',metrics=['acc'])
similarity_model_2.fit_generator(
        (datagen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_shuffled_training_siamese.csv")),
        steps_per_epoch = 102278/32,
        epochs=20,
        validation_data = (valgen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_shuffled_validation_siamese.csv")),
        validation_steps = 5288/32
        )

In [ ]:
df = pd.read_csv(r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_unseen_training_siamese.csv", index_col = 0)
#print(df)
df = np.array(df)
print(df)

def datagen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    #print(df)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\unseen-dataset\\TrainingSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\unseen-dataset\\TrainingSet",df[j][1]),0)
            y = df[j][2]
            #print(loaded_image_1.shape)
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            #print(j)
            if c == batch_size:
                #print(label)
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]
                
def valgen(batch_size, path):
    df = pd.read_csv(path, index_col = 0)
    df = np.array(df)
    j = 0
    while True:
        c = 0
        left = []
        right = []
        label = []
        for i in range(0,batch_size):
            loaded_image_1 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\unseen-dataset\\ValidationSet",df[j][0]),0)
            loaded_image_2 = cv2.imread(os.path.join("C:\\Users\\yashr\\Desktop\\Project2\\unseen-dataset\\ValidationSet",df[j][1]),0)
            y = df[j][2]
            left.append(255.0-loaded_image_1.reshape((64,64,1)))
            right.append(255.0-loaded_image_2.reshape((64,64,1)))
            label.append(y)
            c = c + 1
            j = j + 1
            if j == len(df):
                j = 0
            if c == batch_size:
                yield [np.array(left)/255.0, np.array(right)/255.0], [label]

In [ ]:
imDim = 64
input_shape  = (imDim,imDim,1)
inp_img = Input(shape = (imDim,imDim,1), name = 'ImageInput')
model = inp_img

#     model = Input(shape=(imDim,imDim,1))
#     model.add(Input(shape = (imDim,imDim,1), name = 'FeatureNet_ImageInput'))
model = Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='valid')(model)
#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model = MaxPooling2D((2,2), padding='valid')(model)
model = Conv2D(64, (3, 3), activation='relu',padding='valid')(model)
#     model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
model = Conv2D(128, (3, 3), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(1, (3, 3), activation='relu',padding='same'))
#     model.add(Conv2D(2, (3, 3), activation='relu',padding='same'))

model = Conv2D(256, (1, 1), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)

model = Conv2D(64, (1, 1), activation='relu',padding='valid')(model)
# model = MaxPooling2D((2,2),padding='valid')(model)
model = Flatten()(model)

feat = Model(inputs=[inp_img], outputs=[model],name = 'Feat_Model')
feat.summary()

left_img = Input(shape = (imDim,imDim,1), name = 'left_img')
right_img = Input(shape = (imDim,imDim,1), name = 'right_img')

left_feats = feat(left_img)
right_feats = feat(right_img)

from keras.layers import concatenate
import random

merged_feats = concatenate([left_feats, right_feats], name = 'concat_feats')
merged_feats = Dense(1024, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(4, activation = 'linear')(merged_feats)
merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(1, activation = 'sigmoid')(merged_feats)
similarity_model_3 = Model(inputs = [left_img, right_img], outputs = [merged_feats], name = 'Similarity_Model_3')

In [ ]:
similarity_model_3.compile(loss="binary_crossentropy",optimizer='Adam',metrics=['acc'])
similarity_model_3.fit_generator(
        (datagen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_unseen_training_siamese.csv")),
        steps_per_epoch = 102278/32,
        epochs=20,
        validation_data = (valgen(32,r"C:\\Users\\yashr\\Desktop\\Project2\\dataset_unseen_validation_siamese.csv")),
        validation_steps = 5288/32
        )